In [115]:
import os

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from openai import OpenAI
import streamlit as st

In [116]:
from openai import OpenAI

client = OpenAI(
    base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d"
)

In [117]:
documents = []

In [118]:
loader = PyPDFLoader("C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/test/labor_low.pdf")
pdf_documents = loader.load()

In [119]:
documents.extend(pdf_documents)

In [120]:
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunk = chunk_splitter.split_documents(documents)

In [121]:
embeddings = HuggingFaceBgeEmbeddings()

In [ ]:
vectordb = Chroma.from_documents(documents=chunk, embedding=embeddings)

In [ ]:
query = "안녕?"

In [ ]:
docs = vectordb.similarity_search(query)

In [ ]:
docs

In [ ]:
print(docs[0].page_content)

# jhgan/ko-sbert-nli 모델

In [123]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/test/labor_low.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, length_function=tiktoken_len)
docs = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name ="jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# texts들에 대해 hf 임베딩 모델로 임베딩 하는 과정 -> 이를 docsearch에 저장
docsearch = Chroma.from_documents(docs, hf)

In [ ]:
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# llm = ChatOpenAI(base_url = "http://sionic.chat:8001/v1",
#     api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
#     model="xionic-ko-llama-3-70b",
#     temperature=0.1)

In [ ]:
# qa = RetrievalQA.from_chain_type(llm = llm,
#                                  chain_type= "stuff",
#                                  retriever = docsearch.as_retriever(
#                                      search_type="mmr",
#                                      search_kwargs={'k':3, 'fetch_k': 10}),
#                                      return_source_documents=True)

In [124]:
query="최저수당 대해 출력해줘"
result = qa(query)

In [125]:
result

{'query': '최저수당 대해 출력해줘',
 'result': 'I apologize, but there is no mention of "최저수당" (minimum wage) in the provided context. The context appears to be related to labor laws and regulations, but it does not specifically mention minimum wage. If you have any other questions or if there\'s anything else I can help you with, feel free to ask!',
 'source_documents': [Document(metadata={'page': 7, 'source': 'C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/test/labor_low.pdf'}, page_content='임금을 보장하여야 한다.\n \n제48조(임금대장 및 임금명세서) ① 사용자는 각 사업장별로 임금대장을 작성하고 임금과 가족수당 계산의 기초가 되\n는 사항, 임금액, 그 밖에 대통령령으로 정하는 사항을 임금을 지급할 때마다 적어야 한다. <개정 2021. 5. 18.>\n② 사용자는 임금을 지급하는 때에는 근로자에게 임금의 구성항목ㆍ계산방법, 제43조제1항 단서에 따라 임금의 일\n부를 공제한 경우의 내역 등 대통령령으로 정하는 사항을 적은 임금명세서를 서면(「전자문서 및 전자거래 기본법」\n제2조제1호에 따른 전자문서를 포함한다)으로 교부하여야 한다.<신설 2021. 5. 18.>\n[제목개정 2021. 5. 18.]\n \n제49조(임금의 시효) 이 법에 따른 임금채권은 3년간 행사하지 아니하면 시효로 소멸한다.\n \n       제4장 근로시간과 휴식\n \n제50조(근로시간) ① 1주 간의 근로시간은 휴게시간을 제외하고 40시간을 초과할 수 없다.\n② 1일의 근로시간은 휴게시간을 제외하고 8시간

In [ ]:
client = OpenAI(
    base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d"
)

In [ ]:
def generate_answer(question, result):
    # 질문과 관련된 문서를 사용하여 RAG를 수행하는 요청
    response = client.chat.completions.create(
        model="xionic-ko-llama-3-70b",
        messages=[
            {"role": "system", "content": "You are an AI assistant. You will be given a task. You must generate a detailed and long answer in korean."},
            {"role": "user", "content": question}
        ],
        result=result
    )
    
    return response

In [ ]:
question = "최저수당이 뭐야?"

In [ ]:
response = generate_answer(question, result)
print(response)

In [ ]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/test/labor_low.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, length_function=tiktoken_len)
docs = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name ="jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# texts들에 대해 hf 임베딩 모델로 임베딩 하는 과정 -> 이를 docsearch에 저장
docsearch = Chroma.from_documents(texts, hf)

In [ ]:
qa = RetrievalQA.from_chain_type(llm = openai,
                                 chain_type= "stuff",
                                 retriever = docsearch.as_retriever(
                                     search_type="mmr",
                                     search_kwargs={'k':3, 'fetch_k': 10}),
                                     return_source_documents=True)

query="1대1 대화하는 법에 대해 설명해줘"
result = qa(query)

In [ ]:
def search_documents(query):
    # 쿼리를 사용하여 문서 검색
    results = db.similarity_search(query)
    
    # 검색 결과 중에서 첫 번째 문서의 내용을 반환
    if results:
        return results[0].page_content
    else:
        return "검색 결과를 찾을 수 없습니다."

# 가상의 챗봇 구현
def chatbot(query):
    # 사용자 질문을 받아서 문서 검색
    answer = search_documents(query)
    return answer

# 사용자 입력 받기
user_query = input("질문을 입력하세요: ")

# 챗봇으로부터 답변 받기
answer = chatbot(user_query)

# 챗봇의 답변 출력
print(answer)


In [ ]:
def create_chain(llm, template_path, output_key):
    return LLMChain(
        llm = llm,
        prompt = ChatPromptTemplate.from_template(
            template=read_prompt_template(template_path)
        ),
        output_key=output_key,
        verbose=True,        
    )

In [ ]:
llm = ChatOpenAI(base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
    model="xionic-ko-llama-3-70b",
    temperature=0.1)

In [ ]:
product_data = create_chain(
    llm = llm,
    template_path=docs,
    output_key="text"
)

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]

model = SentenceTransformer('jhgan/ko-sbert-nli')
embeddings = model.encode(sentences)
print(embeddings)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

loader = PyPDFLoader("C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/test/labor_low.pdf")
pages = loader.load_and_split()

# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, length_function=tiktoken_len)
docs = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name ="jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
ko = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

db = FAISS.from_documents(docs, ko)

query = "근로의 정의를 알려줘"
docs = db.similarity_search(query)
print(docs[0].page_content)

# 기존 베이스라인 모델

In [ ]:
import os
from typing import Dict, List

from langchain.chains import ConversationChain, LLMChain, LLMRouterChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel

In [ ]:
documents = []
loader = PyPDFLoader("C:/Users/SUNJIN/Documents/인턴/03_ideachatbot/test/labor_low.pdf")
pdf_documents = loader.load()
documents.extend(pdf_documents)

In [ ]:
def create_chain(llm, documents, output_key):
    chains = []
    for doc in documents:
        prompt = doc.page_content  # 각 문서의 내용을 prompt로 사용
        chain = LLMChain(
            llm=llm,
            prompt=prompt,
            output_key=output_key,
            verbose=True
        )
        chains.append(chain)
    
    return chains

In [ ]:
llm = ChatOpenAI(base_url = "http://sionic.chat:8001/v1",
    api_key = "934c4bbc-c384-4bea-af82-1450d7f8128d",
    model="xionic-ko-llama-3-70b",
    temperature=0.1)

In [ ]:
rule_2 = create_chain(
    llm=llm,
    documents=documents,
    output_key="text",
)